## PyTorch实现ResNet全系列
用PyTorch实现ResNet，包括ResNet18，ResNet34，ResNet50等

In [3]:
import torch.nn as nn
import torch.utils.model_zoo as model_zoo

In [4]:
__all__=['ResNet','resnet18','resnet34','resnet50','resnet101','resnet152']

In [5]:
#添加预训练权重的URL
model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}

In [6]:
#将卷积层进行一次封装
def conv3x3(in_planes,out_planes,stride=1):
    return nn.Conv2d(in_planes,out_planes,kernel_size=3,stride=stride,padding=1,bias=False)

### 定义Block单元
这里就是这个系列的区别，像resnet18和34这类比较轻小的网络使用的是basic block单元。像50和101这类使用的是BottleBlock这类

#### BasicBlock
基础版的残差模块由两个叠加的3*3卷积组成

In [7]:
class BasicBlock(nn.Module):
    expansion=1
    
    def __init__(self,inplanes,planes,stride=1,downsample=None):
        super(BasicBlock,self).__init__()
        self.conv1=conv3x3(inplanes,planes,stride)
        self.bn1=nn.BatchNorm2d(planes)
        self.relu=nn.ReLU(inplace=True)
        self.conv2=conv3x3(planes,planes)
        self.bn2=nn.BatchNorm2d(planes)
        self.downsample=downsample
        self.stride=stride
    def forward(self,x):
        residual=x#残差单元，就是开始的输入
        
        out=self.conv1(x)
        out=self.bn1(out)
        out=self.relu(out)
        
        out=self.conv2(out)
        out=self.bn2(out)
        
        if self.downsample is not None:
            residual=self.downsample(x)
        
        out+=residual#就这样实现了残差连接，就这么简单
        out=self.relu(out)
        
        return out

#### Bottleneck
这个模块主要是ResNet50和ResNet101使用的，与基础版本不同的是这里有三个卷积，分别是1x1,3x3,1x1分别用来压缩维度，卷积处理，恢复维度，inplane是输入的通道，plane是输出的通道，expansion是对输出通道数的倍乘，在基础版的这个参数为1则可以忽略不计。输出的就是plane。而在bottleneck它的任务就是对通道数进行压缩，再放大，plane不再代表输出的通道数，而是block内部压缩后的通道数，输出通道变为plane*expansion

In [8]:
class Bottleneck(nn.Module):
    expansion=4
    
    def __init__(self,inplanes,planes,stride=1,downsample=None):
        super(Bottleneck,self).__init__()
        self.conv1=nn.Conv2d(inplanes,planes,kernel_size=1,bias=False)
        self.bn1=nn.BatchNorm2d(planes)
        self.conv2=nn.Conv2d(planes,planes,kernel_size=3,stride=stride,padding=1,bias=False)
        self.bn2=nn.BatchNorm2d(planes)
        self.conv3=nn.Conv2d(planes,planes*self.expansion,kernel_size=1,bias=False)
        self.bn3=nn.BatchNorm2d(planes*self.expansion)
        self.relu=nn.ReLU(inplace=True)
        self.downsample=downsample
        self.stride=stride
        
    def forward(self,x):
        residual=x
        
        out=self.conv1(x)
        out=self.bn1(out)
        out=self.relu(out)
        
        out=self.conv2(out)
        out=self.bn2(out)
        out=self.relu(out)
        
        out=self.conv3(out)
        out=self.bn3(out)
        
        if self.downsample is not None:
            residual=self.downsample(x)
        out+=residual
        out=self.relu(out)
        
        return out

### 定义ResNet网络主体
resnet共有5个阶段，其中第一阶段为7x7的卷积处理，stride为2，然后经过池化处理，此时特征图的尺寸为输入的1/4,接下来四个阶段用make_layer函数产生四个layer，需要用户输入每个layer的block数目以及采用的block类型(基础版还是bottleneck版)

In [15]:
class ResNet(nn.Module):
    
    def __init__(self,block,layers,num_classes=1000):
        super(ResNet,self).__init__()
        self.inplaces=64
        self.conv1=nn.Conv2d(3,64,kernel_size=7,stride=2,padding=3,bias=False)
        self.bn1=nn.BatchNorm2d(64)
        self.relu=nn.ReLU(inplace=True)
        self.maxpool=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
        self.layer1=self._make_layer(block,64,layers[0])
        self.layer2=self._make_layer(block,128,layers[1],stride=2)
        self.layer3=self._make_layer(block,256,layers[2],stride=2)
        self.layer4=self._make_layer(block,512,layers[3],stride=2)
        self.avgpool=nn.AvgPool2d(7,stride=1)
        self.fc=nn.Linear(512*block.expansion,num_classes)
        #卷积核以及BN的权值初始化
        for m in self.modules():
            if isinstance(m,nn.Conv2d):
                nn.init.kaiming_normal_(m.weight,mode='fan_out',nonlinearity='relu')#权重初始化
            elif isinstance(m,nn.BatchNorm2d):#BatchNorm2d批规范的初始化
                nn.init.constant_(m.weight,1)
                nn.init.constant_(m.bias,0)
                
    def _make_layer(self,block,planes,blocks,stride=1):
        downsample=None
        if stride!=1 or self.inplaces!=planes*block.expansion:
            downsample=nn.Sequential(
                nn.Conv2d(self.inplaces,planes*block.expansion,
                         kernel_size=1,stride=stride,bias=False),
                nn.BatchNorm2d(planes*block.expansion),
            )
        layers=[]
        layers.append(block(self.inplaces,planes,stride,downsample))
        self.inplaces=planes*block.expansion#这里更改inplaces值
        for i in range(1,blocks):
            layers.append(block(self.inplaces,planes))
        return nn.Sequential(*layers)
    
    def forward(self,x):
        x=self.conv1(x)
        x=self.bn1(x)
        x=self.relu(x)
        x=self.maxpool(x)
        
        x=self.layer1(x)
        x=self.layer2(x)
        x=self.layer3(x)
        x=self.layer4(x)
        
        x=self.avgpool(x)
        x=x.view(x.size(0),-1)
        x=self.fc(x)
        
        return x

In [16]:
def resnet50(pretrained=False,**kwargs):
    model=ResNet(Bottleneck,[3,4,6,3],**kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet50']))
    return model

### 定义各个模型
包括resnet18,resnet34,resnet50,resnet102

In [19]:
#resnet18模型
def resnet18(pretrained=False,**kwargs):
    model=ResNet(BasicBlock,[2,2,2,2],**kwargs)
    
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet18']))
    return model
#resnet34模型
def resnet34(pretrained=False,**kwargs):
    model=ResNet(BasicBlock,[3,4,6,3],**kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet34']))
    return model
#resnet50模型
def resnet50(pretrained=False,**kwargs):
    model=ResNet(Bottleneck,[3,4,6,3],**kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet50']))
    return model

#resnet101模型
def resnet101(pretrained=False,**kwargs):
    model=ResNet(Bottleneck,[3,4,23,3],**kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet101']))
    return model
#resnet152模型
def resnet152(pretrained=False,**kwargs):
    model=ResNet(Bottleneck,[3,8,36,3],**kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet152']))
    return model

In [20]:
resnet18=resnet18()
print(resnet18)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [23]:
!pip install thop

You should consider upgrading via the '/Users/gongpengwang/anaconda/anaconda3/envs/pytorch_py3.6/bin/python -m pip install --upgrade pip' command.


In [26]:
import torch
net=resnet50()
inp=torch.rand(1,3,224,224)
out=net(inp)

In [28]:
print(out.size())

torch.Size([1, 1000])
